In [3]:
import wandb
import json
import os, sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir)))
from utils import _scale_params, load_config

/home/pelinski/Dropbox/phd/projects/faab/.venv/lib/python3.10/site-packages/wandb/sdk/launch/builder/build.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [ ]:
api = wandb.Api()
api.entity = "faab-hyperparams"


In [ ]:

sweep_ids = ["tonczgio",
"llctcl68",
"e33s48ug",
"tmuf8nz6",
"shen1jqe",
"12ie0tmu",
"f8ayvluc",
"7cl8otxa",
"rxqbfbtd",
"p0cx9m4p"]
sweeps = [api.sweep(f'faab-hyperparams/faab_autoencoder_minidataset/{sweep}') for sweep in sweep_ids]
_path = "../trained/transformer-minidataset"


In [3]:
# finished_runs = []

# # Iterate over runs in the sweep
# for run in sweep.runs:
#     # Fetch the run to get detailed information, including its state
#     detailed_run = wandb.Api().run(f"{run.project}/{run.id}")
#     # Check if the run is finished
#     if detailed_run.state == "finished":
#         finished_runs.append(detailed_run)


In [4]:
# import os

# #load api key from .env
# with open("../.env") as f:
#     for line in f:
#         if line.strip() and not line.startswith('#'):
#             key, value = line.strip().split('=', 1)
#             os.environ[key] = value


In [5]:
# wandb.login(key=os.environ["WANDB_API_KEY"])

In [5]:
id_ep = {}
run_ids = []
for sweep in sweeps:
    for run in sweep.runs:
        files = [file.name for file in run.files()]
        matching_files = [file for file in files if file.endswith(".model")]
        epochs = [int(file.split("_")[-1].split(".model")[0]) for file in matching_files]
        if epochs == []:
            continue
        highest_epoch = max(epochs)
        id_ep[run.id] = highest_epoch
        run_ids.append(run.id)
        print(run.id, highest_epoch)    
        root_file_name = f'transformer_run_{run.id}_{highest_epoch}'
        run.file(f'{root_file_name}.model').download(root=_path, exist_ok=True)
        json.dump(run.config, open(f'{_path}/{root_file_name}.json', "w"))
        json.dump({"train_loss": run.summary["train_loss"]}, open(f'{_path}/{root_file_name}_metrics.json', "w"))

json.dump(run_ids, open(f'{_path}/run_ids.json', "w"))

bqgct1xf 75
8ci602ed 75
wy9m78x6 75
qqf3nym9 75
42jy2fxw 75
cco08cwt 75
rj02z1yf 75
5h6gpgli 75
wecyjjv6 75
a9wnfeis 75
guca3j3n 75
zte64g1k 75
d4p9psev 75
i9m7wsct 75
5qndphh8 75
5k1838fm 75
x83w54gk 75
i8dajq5v 75
rarx6o1s 75
l8wab3qb 75
f0i9vau9 75
j9980erp 75
9b58xj1e 75
qfqaf3a0 75
96rsdi7g 75
hujc76vx 75
goylnvgv 25
mh9u1uwt 75
z4q6c7dy 75
j1kfzisy 75
cgy6m30d 75
ljwe16mw 75
kk5b7b44 75
0lhbjjox 75
r2r9n709 75
b4gmn89t 75
yub5cwga 75
jq8ts7qg 75
4hwr1dmc 75
qdrpgjv7 75
4eyhw9l5 75
jw8cws22 75
pbok65yh 75
21soegq8 75
243x9axq 75
2b9nn4w4 75
wctedkaq 75
zpiylfsf 75
9f85pw8o 75
e251ayzv 75
tmq9segx 75
g85pwm7w 75
sh3yox5o 75
4kpv5ny6 75
eqgrcsfz 75
089qtbq9 75
xzkrrlon 75
1qvcwtuy 75
h98zxhqo 75
3ciea2fd 75
wwyb5emc 75
5d97cdth 75
49a2ejsd 75
1vfuf4yf 75
9j4b7ykt 75
vfq0mqr2 75
e2gt57qi 75
2eg5lihp 75
b70ksuej 75
8zn32fpo 75
5yvxxehz 75
d28qmw4p 50
9ex6w4o7 50
7a75g94k 75
pr6zwy0y 75
x335qqnn 75
ppde8iak 75
ionnpajj 75
bhtqerw4 75
ov2quu9v 75
2ovhhhq4 75
xh2luxo1 75
hmemhukc 75
xoa9

In [57]:
_path = "../trained/transformer-minidataset"


In [8]:
json.dump(id_ep, open(f'{_path}/id_ep.json', "w"))

In [9]:
_train_losses = {}
for _id in run_ids:
    _train_losses[_id] = json.load(open(f'{_path}/transformer_run_{_id}_{id_ep[_id]}_metrics.json'))["train_loss"]

models_ordered_by_asc_loss = list(dict(sorted(_train_losses.items(), key=lambda item: item[1])).keys())
filename = _path + "/models_ordered_by_asc_loss.json"

with open(filename, 'w') as file:
    json.dump(models_ordered_by_asc_loss, file)

In [5]:
# load id_ep
id_ep = json.load(open(f'{_path}/id_ep.json'))

In [6]:
scaled_model_coordinates = _scale_params(epochs=id_ep, path = _path)

filename = _path + "/scaled_params.json"
with open (filename, 'w') as file:
    json.dump(scaled_model_coordinates, file)

In [7]:
all_configs = {}
run_ids = json.load(open(f'{_path}/run_ids.json'))
for run in run_ids:
    config = load_config(run, epochs=id_ep[run],path=_path)
    all_configs[run] = config
    
    

In [8]:
all_configs

{'bqgct1xf': {'mask': False,
  'pred': False,
  'model': 'transformer',
  'device': 'cuda:0',
  'epochs': 76,
  'd_model': 4,
  'dropout': 0.2855777170478696,
  'ff_size': 64,
  'project': 'faab_autoencoder_minidataset',
  'seq_len': 1024,
  'tf_type': 'timelin',
  'criterion': 'weighted_mse',
  'num_heads': 2,
  'optimizer': 'rmsprop',
  'batch_size': 32,
  'experiment': 'multiphonics',
  'num_layers': 5,
  'pickle_path': 'src/dataset/data/0814/multiphonics/processed.pkl',
  'feat_in_size': 8,
  'weights_path': 'src/dataset/data/0814/multiphonics/weights.pkl',
  'learning_rate': 0.00023275188241908096,
  'max_grad_norm': 0,
  'pe_scale_factor': 1,
  'plotter_samples': 5,
  'scheduler_gamma': 0.1,
  'num_encoder_layers': 7,
  'scheduler_step_size': 0},
 '8ci602ed': {'mask': False,
  'pred': False,
  'model': 'transformer',
  'device': 'cuda:0',
  'epochs': 76,
  'd_model': 4,
  'dropout': 0.10540070160110164,
  'ff_size': 128,
  'project': 'faab_autoencoder_minidataset',
  'seq_len': 1

In [9]:
len(run_ids)

204

In [10]:
all_configs 

{'bqgct1xf': {'mask': False,
  'pred': False,
  'model': 'transformer',
  'device': 'cuda:0',
  'epochs': 76,
  'd_model': 4,
  'dropout': 0.2855777170478696,
  'ff_size': 64,
  'project': 'faab_autoencoder_minidataset',
  'seq_len': 1024,
  'tf_type': 'timelin',
  'criterion': 'weighted_mse',
  'num_heads': 2,
  'optimizer': 'rmsprop',
  'batch_size': 32,
  'experiment': 'multiphonics',
  'num_layers': 5,
  'pickle_path': 'src/dataset/data/0814/multiphonics/processed.pkl',
  'feat_in_size': 8,
  'weights_path': 'src/dataset/data/0814/multiphonics/weights.pkl',
  'learning_rate': 0.00023275188241908096,
  'max_grad_norm': 0,
  'pe_scale_factor': 1,
  'plotter_samples': 5,
  'scheduler_gamma': 0.1,
  'num_encoder_layers': 7,
  'scheduler_step_size': 0},
 '8ci602ed': {'mask': False,
  'pred': False,
  'model': 'transformer',
  'device': 'cuda:0',
  'epochs': 76,
  'd_model': 4,
  'dropout': 0.10540070160110164,
  'ff_size': 128,
  'project': 'faab_autoencoder_minidataset',
  'seq_len': 1

In [14]:
# select 5 top models from every experiment
experiments = ["drone_example", "feedback_bloom", "multiphonics", "pizzicato_harmonics", "waveset_harmonics"]

exp_runs = {exp: [] for exp in experiments}
for exp in experiments:
    # get all runs for experiment
    for run in run_ids:
        if all_configs[run]["experiment"] == exp:
            exp_runs[exp].append(run)


In [17]:
# get top 5 runs for each experiment

_train_losses = {}
for _id in run_ids:
    _train_losses[_id] = json.load(open(f'{_path}/transformer_run_{_id}_{id_ep[_id]}_metrics.json'))["train_loss"]
    
top_runs = {exp: [] for exp in experiments}
for exp in experiments:
    # sort runs by train_loss
    sorted_runs = sorted(exp_runs[exp], key=lambda x: _train_losses[x])
    top_runs[exp] = sorted_runs[:5]

In [18]:
top_runs

{'drone_example': ['7jek9blg', 'xt1b5ncs', 'tbimy590', 'kdfu69md', 'ov2quu9v'],
 'feedback_bloom': ['21soegq8',
  '0lhbjjox',
  '1qvcwtuy',
  'b4gmn89t',
  'jw8cws22'],
 'multiphonics': ['rarx6o1s', 'qfqaf3a0', '5qndphh8', 'a9wnfeis', 'awaen0te'],
 'pizzicato_harmonics': ['8wzblws2',
  '2s5lgg5h',
  'e7pccu76',
  'u9xd4jpq',
  '7wvj11dh'],
 'waveset_harmonics': ['56lvlmuj',
  'gza0sltm',
  'r21hwu9p',
  '7f4fvk0u',
  'rlx8q7wd']}

In [27]:

# top runs ordered by asc loss
all_top_runs = [run for exp in experiments for run in top_runs[exp]]
top_runs_ordered_by_asc_loss = sorted(all_top_runs, key=lambda x: _train_losses[x])

# save top runs ordered by asc loss
filename = _path + "/top-runs/models_ordered_by_asc_loss.json"
with open(filename, 'w') as file:
    json.dump(top_runs_ordered_by_asc_loss, file)

In [21]:
# copy to runs to new folder
import shutil

for exp in experiments:
    for run in top_runs[exp]:
        src_model = f'{_path}/transformer_run_{run}_{id_ep[run]}.model'
        src_config = f'{_path}/transformer_run_{run}_{id_ep[run]}.json'
        src_metrics = f'{_path}/transformer_run_{run}_{id_ep[run]}_metrics.json'
        
        dest_model = f'{_path}/top-runs/transformer_run_{run}_{id_ep[run]}.model'
        dest_config = f'{_path}/top-runs/transformer_run_{run}_{id_ep[run]}.json'
        dest_metrics = f'{_path}/top-runs/transformer_run_{run}_{id_ep[run]}_metrics.json'
        
        os.makedirs(os.path.dirname(dest_model), exist_ok=True)
        shutil.copy(src_model, dest_model)
        shutil.copy(src_config, dest_config)
        shutil.copy(src_metrics, dest_metrics)

In [29]:
#id_ep for top runs
top_runs_id_ep = {run: id_ep[run] for exp in experiments for run in top_runs[exp]}
filename = _path + "/top-runs/id_ep.json"
with open(filename, 'w') as file:
    json.dump(top_runs_id_ep, file)

In [37]:
# run ids for top runs
top_run_ids = []
for exp in experiments:
    for run in top_runs[exp]:
        top_run_ids.append(run)

filename = _path + "/top-runs/run_ids.json"
with open(filename, 'w') as file:
    json.dump(top_run_ids, file)

In [58]:
# scaled params for top runs
scaled_model_coordinates_top_runs = {run: scaled_model_coordinates[run] for exp in experiments for run in top_runs[exp]}

filename = _path + "/top-runs/scaled_params.json"
with open(filename, 'w') as file:
    json.dump(scaled_model_coordinates_top_runs, file)

In [49]:
#copy 5 random runs from each experiment 

import random
random_runs = {exp: [] for exp in experiments}

# select 5 random runs from each experiment
for exp in experiments:
    random_runs[exp] = random.sample(exp_runs[exp], 5)

In [51]:
# copy to runs to new folder
import shutil

for exp in experiments:
    for run in random_runs[exp]:
        src_model = f'{_path}/all/transformer_run_{run}_{id_ep[run]}.model'
        src_config = f'{_path}/all/transformer_run_{run}_{id_ep[run]}.json'
        src_metrics = f'{_path}/all/transformer_run_{run}_{id_ep[run]}_metrics.json'
        
        dest_model = f'{_path}/random-runs/transformer_run_{run}_{id_ep[run]}.model'
        dest_config = f'{_path}/random-runs/transformer_run_{run}_{id_ep[run]}.json'
        dest_metrics = f'{_path}/random-runs/transformer_run_{run}_{id_ep[run]}_metrics.json'
        
        os.makedirs(os.path.dirname(dest_model), exist_ok=True)
        
        shutil.copy(src_model, dest_model)
        shutil.copy(src_config, dest_config)
        shutil.copy(src_metrics, dest_metrics)

In [52]:
# random runs ordered by asc loss
all_random_runs = [run for exp in experiments for run in random_runs[exp]]
random_runs_ordered_by_asc_loss = sorted(all_random_runs, key=lambda x: _train_losses[x])

# save random runs ordered by asc loss
filename = _path + "/random-runs/models_ordered_by_asc_loss.json"
with open(filename, 'w') as file:
    json.dump(random_runs_ordered_by_asc_loss, file)

In [53]:
#id_ep for random runs
random_runs_id_ep = {run: id_ep[run] for exp in experiments for run in random_runs[exp]}
filename = _path + "/random-runs/id_ep.json"
with open(filename, 'w') as file:
    json.dump(random_runs_id_ep, file)

In [59]:
# scaled params for random runs
scaled_model_coordinates_random_runs = {run: scaled_model_coordinates[run] for exp in experiments for run in random_runs[exp]}

filename = _path + "/random-runs/scaled_params.json"
with open(filename, 'w') as file:
    json.dump(scaled_model_coordinates_random_runs, file)

In [55]:
random_runs

{'drone_example': ['x086wfiq', 'u3gpoeb5', 'tbimy590', 'tcsffrxh', 'kdfu69md'],
 'feedback_bloom': ['b70ksuej',
  'jw8cws22',
  'zpiylfsf',
  'z4q6c7dy',
  '49a2ejsd'],
 'multiphonics': ['x46ov0fl', 'j9980erp', 'rj02z1yf', 'me1o4o0n', '7gxqitbr'],
 'pizzicato_harmonics': ['h5km9t5p',
  'yicubzho',
  '2s5lgg5h',
  '0s44sabe',
  'v0i1bb6l'],
 'waveset_harmonics': ['c3xau2c1',
  'lqaq5o7t',
  'ig61duyp',
  '1kr3jr21',
  '7f4fvk0u']}

In [56]:
# run ids for random runs
random_run_ids = []
for exp in experiments:
    for run in random_runs[exp]:
        random_run_ids.append(run)

filename = _path + "/random-runs/run_ids.json"
with open(filename, 'w') as file:
    json.dump(random_run_ids, file)

In [44]:
api = wandb.Api()
api.entity = "faab-hyperparams"

run = "vfq0mqr2"
run = api.run(f"faab-hyperparams/faab_autoencoder_minidataset/{run}")
root_file_name = f'transformer_run_{run.id}_{id_ep[run.id]}'
_path = "../trained/transformer-minidataset/random-runs"
run.file(f'{root_file_name}.model').download(root=_path, exist_ok=True)


<_io.TextIOWrapper name='../trained/transformer-minidataset/random-runs/transformer_run_vfq0mqr2_75.model' mode='r' encoding='UTF-8'>